In [1]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup import setup_2,setup_1
import warnings
from datetime import datetime,timedelta
warnings.filterwarnings("ignore")

In [33]:
pontos = 50
rate_stop = 1
rate_tp = 3
spread = 5
var_bb = 2

timeframe = 5
                
ativo = f'win_{timeframe}_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[(df['Data'] > '2022-11-01') & (df['Data'] < '2022-12-30')]

df_ticks = pd.read_csv('./Dados/tick_win_11_12.csv')

df_acao = setup_2(df,df_ticks,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp,timeframe = timeframe)


In [34]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')


Timeframe: 5 --pontos: 50 --- rate_tp: 3 -- var_bb: 2
Pontos_liquido: 8090.0 -- Total_operacoes: 162
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09             24     0.500000          1080.0
1     10             19     0.578947          1155.0
2     11             14     0.571429           830.0
3     12             12     0.750000          1140.0
4     13             15     0.533333           775.0
5     14             16     0.437500           520.0
6     15             20     0.600000          1300.0
7     16             17     0.411765           465.0
8     17             19     0.421053           555.0
9     18              5     0.600000           325.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  11             87     0.540230          4615.0
1  12             74     0.513514          3530.0



In [32]:
df_acao.tail(10)

,Data,Open,Close,acao,stop,tp,resultado_binario,Hora,Hora_h,Mes
99755,2022-12-20 17:00:00,108790.0,108635.0,call,108585.0,108785.0,1.0,17:00:00,17,12
99769,2022-12-20 18:10:00,108945.0,109030.0,sell,109080.0,108880.0,0.0,18:10:00,18,12
99786,2022-12-21 10:10:00,109420.0,109685.0,sell,109735.0,109535.0,0.0,10:10:00,10,12
99826,2022-12-21 13:30:00,109055.0,109360.0,sell,109410.0,109210.0,0.0,13:30:00,13,12
99851,2022-12-21 15:35:00,109020.0,109165.0,sell,109215.0,109015.0,0.0,15:35:00,15,12
99877,2022-12-21 17:45:00,109290.0,109375.0,sell,109425.0,109225.0,1.0,17:45:00,17,12
99896,2022-12-22 09:55:00,109380.0,109550.0,sell,109600.0,109400.0,0.0,09:55:00,09,12
99910,2022-12-22 11:05:00,109190.0,109005.0,call,108955.0,109155.0,1.0,11:05:00,11,12
99953,2022-12-22 14:40:00,109125.0,109065.0,call,109015.0,109215.0,1.0,14:40:00,14,12
99984,2022-12-22 17:15:00,108900.0,109115.0,sell,109165.0,108965.0,0.0,17:15:00,17,12


In [ ]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes


df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

df_total = pd.concat([df_total,df_new])

print(df_total.to_string())